In [1]:
import numpy as np
from functools import lru_cache

In [2]:
forms_fp = "./rubiks-example.txt"

In [3]:
forms = open(forms_fp, 'r').read()
forms_dict = {}
for i,form in enumerate(forms.split(':')[1:]):
  forms_dict[i] = form.split('Form')[0]

# New Section

In [4]:
class RubiksHandler():

  def __init__(self,
               tensor = None,
               text = None,
               canonical_form: dict = {'top':    'Y',
                                       'left':   'G',
                                       'front':  'W',
                                       'right':  'O',
                                       'back':   'B',
                                       'bottom': 'R'
                                      },
              ):
    self.canonical_form = canonical_form
    if tensor==None and text==None:
      raise "Must provide atleast one, tensor or txt"

    if  not tensor:
      self.text = text
      self.tensor = self._text2tensor()
    else:
      self.tensor = tensor
      self.text = self._tensor2text()

    self.top,self.front,self.left,self.right,self.back,self.bottom = self.tensor

  def update(self):
    self.tensor = np.array([self.top,self.front,self.left,self.right,self.back,self.bottom])
    self.text = self._tensor2text()

  def toCanonical(self):
    currForm = self.currForm()    
    relationMap = {
        currForm['top']:self.canonical_form['top'],
        currForm['left']:self.canonical_form['left'],
        currForm['front']:self.canonical_form['front'],
        currForm['right']:self.canonical_form['right'],
        currForm['back']:self.canonical_form['back'],
        currForm['bottom']:self.canonical_form['bottom']
        }
    self.translate(relationMap)

  def _text2tensor(self):
    txt = self.text
    top = [[],[],[]]
    top_c = txt[9:44].split()
    top[0] = top_c[:3]
    top[1] = top_c[3:6]
    top[2] = top_c[6:9]

    front = [[],[],[]]
    front[0] = txt[54:59].split()
    front[1] = txt[81:86].split()
    front[2] = txt[108:113].split()

    left = [[],[],[]]
    left[0] = txt[48:53].split()
    left[1] = txt[75:80].split()
    left[2] = txt[102:107].split()

    right = [[],[],[]]
    right[0] = txt[60:65].split()
    right[1] = txt[87:92].split()
    right[2] = txt[114:119].split()

    back = [[],[],[]]
    back[0] = txt[66:71].split()
    back[1] = txt[93:98].split()
    back[2] = txt[120:125].split()

    bottom = [[],[],[]]
    bottom_c = txt[135:170].split()
    bottom[0] = bottom_c[:3]
    bottom[1] = bottom_c[3:6]
    bottom[2] = bottom_c[6:9]

    return np.array([top,front,left,right,back,bottom])
  
  def _tensor2text(self):

    empty = [' ',' ',' ']
    top, front, left, right, back, bottom = self.tensor
    
    text  =  ' '.join(empty)   + ' ' + ' '.join(top[0])    + ' ' + ' '.join(empty)    + ' ' + ' '.join(empty)   + '\n'
    text +=  ' '.join(empty)   + ' ' + ' '.join(top[1])    + ' ' + ' '.join(empty)    + ' ' + ' '.join(empty)   + '\n'
    text +=  ' '.join(empty)   + ' ' + ' '.join(top[2])    + ' ' + ' '.join(empty)    + ' ' + ' '.join(empty)   + '\n'
    text +=  ' '.join(left[0]) + ' ' + ' '.join(front[0])  + ' ' + ' '.join(right[0]) + ' ' + ' '.join(back[0]) + '\n'
    text +=  ' '.join(left[1]) + ' ' + ' '.join(front[1])  + ' ' + ' '.join(right[1]) + ' ' + ' '.join(back[1]) + '\n'
    text +=  ' '.join(left[2]) + ' ' + ' '.join(front[2])  + ' ' + ' '.join(right[2]) + ' ' + ' '.join(back[2]) + '\n'
    text +=  ' '.join(empty)   + ' ' + ' '.join(bottom[0]) + ' ' + ' '.join(empty)    + ' ' + ' '.join(empty)   + '\n'
    text +=  ' '.join(empty)   + ' ' + ' '.join(bottom[1]) + ' ' + ' '.join(empty)    + ' ' + ' '.join(empty)   + '\n'
    text +=  ' '.join(empty)   + ' ' + ' '.join(bottom[2]) + ' ' + ' '.join(empty)    + ' ' + ' '.join(empty)   + '\n'

    return text
  
  def translate(self, relationMap):
    self.text = self.text.translate(str.maketrans(relationMap))
    self.tensor = self._text2tensor()

  def _findFaceColor(self, face):
    return face[1,1]

  def currForm(self):
    top, front, left, right, back, bottom = self.tensor
    d = {
        'top':    self._findFaceColor(top),
        'front':  self._findFaceColor(front),
        'left':   self._findFaceColor(left),
        'right':  self._findFaceColor(right),
        'back':   self._findFaceColor(back),
        'bottom': self._findFaceColor(bottom)
        }

    return d

In [5]:
print(forms_dict[0])


        Y Y R 
        G Y O 
        O R Y 
  W W W R W W O R W G B G 
  O G G B W R B O Y O B B 
  R G R O R B Y O B O W G 
        B G G 
        Y R W 
        B Y Y 



In [6]:
out_str = ""
for i in range(1,len(forms_dict)):
   rh = RubiksHandler(text = forms_dict[i])
   rh.toCanonical()
   out_str += f"\nForm {i}:\n"
   out_str += "Original Form:\n"
   out_str += forms_dict[i]
   out_str += "\nCanonical Form:\n"
   out_str += rh.text
output = open("rubiks-output.txt","w")
output.write(out_str)
output.close()
print(out_str[:10000])


Form 1:
Original Form:

        R G G 
        B O W 
        R B B 
  O G O G W Y R Y B R O Y 
  G G Y R R Y B Y R O W R 
  W W W G R G W O Y W W O 
        B B O 
        G B Y 
        Y O B 

Canonical Form:

        W G G 
        R Y B 
        W R R 
  Y G Y G B O W O R W Y O 
  G G O W W O R O W Y B W 
  B B B G W G B Y O B B Y 
        R R Y 
        G R O 
        O Y R 

Form 2:
Original Form:

        Y R B 
        G W W 
        W B R 
  G W R Y G B O Y R W W O 
  R R G Y Y G O G R Y B B 
  R O W G B W B R B O B O 
        Y O G 
        Y O O 
        Y W G 

Canonical Form:

        W G B 
        O Y Y 
        Y B G 
  O Y G W O B R W G Y Y R 
  G G O W W O R O G W B B 
  G R Y O B Y B G B R B R 
        W R O 
        W R R 
        W Y O 

Form 3:
Original Form:

        W Y Y 
        O G B 
        W O O 
  G Y G Y B R W R O W G R 
  Y Y R W W R O R W G B W 
  B B B Y W Y B G R B B G 
        O O G 
        Y O R 
        R G O 

Canonical Form:

        W G G 
 